# P4 Regularización, Normalización y Aumentado de datos



In [1]:
## Importar y normalizar datos

from tensorflow import keras
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('training set', x_train.shape)
print('test set', x_test.shape)

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize [0..255]-->[0..1]
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
num_classes=10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

print('training set', x_train.shape)
print('val set', x_val.shape)

2023-11-15 17:43:47.678877: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-15 17:43:47.730721: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-15 17:43:47.915319: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-15 17:43:47.915347: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-15 17:43:47.916463: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

training set (60000, 28, 28)
test set (10000, 28, 28)
training set (48000, 784)
val set (12000, 784)


## Modelo base
 Partiremos de una topología base e iremos añadiendo diferentes estrategias de regularización para mejorar el rendimiento del modelo.

 

In [2]:
from keras import Sequential
from keras.layers import Dense, Input
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau

model = Sequential()

model.add(Input(784))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

opt=SGD(learning_rate=0.025, momentum=0.9)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                patience=2, min_lr=0.00001)
checkpoint = keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)


epochs=25
batch_size=128
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[reduce_lr,checkpoint])  

## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/25
371/375 [============================>.] - ETA: 0s - loss: 0.3417 - accuracy: 0.9010
Epoch 1: val_accuracy improved from -inf to 0.95133, saving model to best_model.h5
375/375 [==============================] - 4s 9ms/step - loss: 0.3399 - accuracy: 0.9016 - val_loss: 0.1709 - val_accuracy: 0.9513 - lr: 0.0250
Epoch 2/25
 23/375 [>.............................] - ETA: 2s - loss: 0.1529 - accuracy: 0.9558

/home/pegi/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


371/375 [============================>.] - ETA: 0s - loss: 0.1347 - accuracy: 0.9606
Epoch 2: val_accuracy improved from 0.95133 to 0.96083, saving model to best_model.h5
375/375 [==============================] - 3s 8ms/step - loss: 0.1344 - accuracy: 0.9606 - val_loss: 0.1289 - val_accuracy: 0.9608 - lr: 0.0250
Epoch 3/25
375/375 [==============================] - ETA: 0s - loss: 0.0892 - accuracy: 0.9742
Epoch 3: val_accuracy improved from 0.96083 to 0.96967, saving model to best_model.h5
375/375 [==============================] - 3s 8ms/step - loss: 0.0892 - accuracy: 0.9742 - val_loss: 0.1051 - val_accuracy: 0.9697 - lr: 0.0250
Epoch 4/25
375/375 [==============================] - ETA: 0s - loss: 0.0635 - accuracy: 0.9814
Epoch 4: val_accuracy improved from 0.96967 to 0.97217, saving model to best_model.h5
375/375 [==============================] - 3s 8ms/step - loss: 0.0635 - accuracy: 0.9814 - val_loss: 0.0893 - val_accuracy: 0.9722 - lr: 0.0250
Epoch 5/25
372/375 [=============

## Regularización l2 (o l1)

La regularización l2 consiste en añadir a la función de coste una penalización proporcional a la norma l2 de los pesos del modelo. De esta forma, se penaliza a los pesos que tengan un valor alto, forzando a que los pesos tengan valores pequeños. Esto se conoce como regularización l2. También podríamos hacer lo mismo con regularización l1 o con ambas (lo que se conoce como *Elastic net*)


In [3]:
## Teniendo en cuenta el modelo base añade regularización L2 a las capas densas
from keras import Sequential
from keras.layers import Dense, Input
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau
from keras.regularizers import l2

model = Sequential()

model.add(Input(784))
model.add(Dense(1024, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dense(1024, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dense(num_classes, activation='softmax'))

opt=SGD(learning_rate=0.025, momentum=0.9)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                patience=2, min_lr=0.00001)
checkpoint = keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)


epochs=25
batch_size=128
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[reduce_lr,checkpoint])  

## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/25
375/375 [==============================] - ETA: 0s - loss: 5.8529 - accuracy: 0.8834
Epoch 1: val_accuracy improved from -inf to 0.92217, saving model to best_model.h5
375/375 [==============================] - 4s 10ms/step - loss: 5.8529 - accuracy: 0.8834 - val_loss: 0.9253 - val_accuracy: 0.9222 - lr: 0.0250
Epoch 2/25
373/375 [============================>.] - ETA: 0s - loss: 0.5756 - accuracy: 0.9292
Epoch 2: val_accuracy improved from 0.92217 to 0.94008, saving model to best_model.h5
375/375 [==============================] - 4s 10ms/step - loss: 0.5746 - accuracy: 0.9294 - val_loss: 0.4420 - val_accuracy: 0.9401 - lr: 0.0250
Epoch 3/25
372/375 [============================>.] - ETA: 0s - loss: 0.4281 - accuracy: 0.9393
Epoch 3: val_accuracy improved from 0.94008 to 0.94217, saving model to best_model.h5
375/375 [==============================] - 4s 11ms/step - loss: 0.4279 - accuracy: 0.9394 - val_loss: 0.4156 - val_accuracy: 0.9422 - lr: 0.0250
Epoch 4/25
375/375 [==

## Dropout

El dropout es una técnica de regularización que consiste en eliminar aleatoriamente un porcentaje de las neuronas de la red durante el entrenamiento. De esta forma, se evita que la red se sobreajuste a los datos de entrenamiento y se mejora la generalización del modelo.


In [4]:
## Teniendo en cuenta el modelo base añade regularización de tipo dropout a las capas densas
from keras import Sequential
from keras.layers import Dense, Input, Dropout
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau
from keras.regularizers import l2

model = Sequential()

model.add(Input(784))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

opt=SGD(learning_rate=0.025, momentum=0.9)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                patience=2, min_lr=0.00001)
checkpoint = keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)


epochs=25
batch_size=128
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[reduce_lr,checkpoint])  

## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Epoch 1/25
375/375 [==============================] - ETA: 0s - loss: 0.4471 - accuracy: 0.8617
Epoch 1: val_accuracy improved from -inf to 0.94692, saving model to best_model.h5
375/375 [==============================] - 4s 10ms/step - loss: 0.4471 - accuracy: 0.8617 - val_loss: 0.1717 - val_accuracy: 0.9469 - lr: 0.0250
Epoch 2/25
372/375 [============================>.] - ETA: 0s - loss: 0.1925 - accuracy: 0.9422
Epoch 2: val_accuracy improved from 0.94692 to 0.96108, saving model to best_model.h5
375/375 [==============================] - 4s 10ms/step - loss: 0.1921 - accuracy: 0.9423 - val_loss: 0.1273 - val_accuracy: 0.9611 - lr: 0.0250
Epoch 3/25
373/375 [============================>.] - ETA: 0s - loss: 0.1460 - accuracy: 0.9549
Epoch 3: val_accuracy improved from 0.96108 to 0.96917, saving model to best_model.h5
375/375 [==============================] - 4s 10ms/step - loss: 0.1457 - accuracy: 0.9550 - val_loss: 0.1026 - val_accuracy: 0.9692 - lr: 0.0250
Epoch 4/25
372/375 [==

## Normalización BatchNorm

La normalización BatchNorm consiste en normalizar la salida de una capa de la red neuronal para que tenga media 0 y varianza 1. De esta forma, se consigue que la red neuronal pueda entrenarse más rápido y que sea más robusta a cambios en los pesos de las capas anteriores.


In [5]:
## Teniendo en cuenta el modelo base añade normalización BatchNorm
from keras import Sequential
from keras.layers import Dense, Input, Dropout, BatchNormalization
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau

model = Sequential()

model.add(Input(784))
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

opt=SGD(learning_rate=0.025, momentum=0.9)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                patience=2, min_lr=0.00001)
checkpoint = keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)


epochs=25
batch_size=128
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[reduce_lr,checkpoint])  

## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Epoch 1/25
370/375 [============================>.] - ETA: 0s - loss: 0.2096 - accuracy: 0.9403
Epoch 1: val_accuracy improved from -inf to 0.96333, saving model to best_model.h5
375/375 [==============================] - 5s 11ms/step - loss: 0.2080 - accuracy: 0.9406 - val_loss: 0.1223 - val_accuracy: 0.9633 - lr: 0.0250
Epoch 2/25
374/375 [============================>.] - ETA: 0s - loss: 0.0647 - accuracy: 0.9795
Epoch 2: val_accuracy improved from 0.96333 to 0.97208, saving model to best_model.h5
375/375 [==============================] - 4s 11ms/step - loss: 0.0647 - accuracy: 0.9795 - val_loss: 0.1025 - val_accuracy: 0.9721 - lr: 0.0250
Epoch 3/25
372/375 [============================>.] - ETA: 0s - loss: 0.0363 - accuracy: 0.9876
Epoch 3: val_accuracy improved from 0.97208 to 0.97600, saving model to best_model.h5
375/375 [==============================] - 4s 11ms/step - loss: 0.0364 - accuracy: 0.9876 - val_loss: 0.0838 - val_accuracy: 0.9760 - lr: 0.0250
Epoch 4/25
374/375 [==

## Aumentado de datos

El aumentado de datos consiste en generar nuevos datos de entrenamiento a partir de los datos de entrenamiento originales. De esta forma, se consigue que el modelo sea más robusto y que se generalice mejor a datos que no ha visto durante el entrenamiento.

En nuestro caso para los dígitos de la MNIST vamos a realizar un aumento de datos de la siguiente forma:

- Rotación aleatoria de la imagen entre -30 y 30 grados.
- Traslación aleatoria de la imagen entre -3 y 3 píxeles en horizontal y vertical.
- Escalado aleatorio de la imagen entre 0.8 y 1.2.
- Inversión aleatoria de la imagen en horizontal y vertical. **NO!!!**

El aumentado de datos se ejecuta en CPU y ralentiza el entrenamiento.

Normalmente además, se necesitarán más epochs para entrenar el modelo. 

In [6]:
## Implementamos en el ejemplo base el aumentado de datos
from keras import Sequential
from keras.layers import Dense, Input, Dropout, BatchNormalization,Reshape
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau

from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest')

## Importante: ImageDataGenerator espera una imagen con 3 canales, necesitamos hacer reshape
x_train = x_train.reshape(48000, 28, 28, 1)
x_val = x_val.reshape(12000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)

## Ajustamos el generador de datos
datagen.fit(x_train)

from keras import Sequential
from keras.layers import Dense, Input, Dropout, BatchNormalization
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau

model = Sequential()

model.add(Input((28,28,1)))
model.add(Reshape((784,)))
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

opt=SGD(learning_rate=0.025, momentum=0.9)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=2, min_lr=0.00001)
checkpoint = keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)


epochs=25
batch_size=128
## Entrenamos con el generador de datos en lugar de con el dataset
history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[reduce_lr,checkpoint])

## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])




Epoch 1/25
374/375 [============================>.] - ETA: 0s - loss: 0.5958 - accuracy: 0.8218
Epoch 1: val_accuracy improved from -inf to 0.94075, saving model to best_model.h5
375/375 [==============================] - 12s 30ms/step - loss: 0.5952 - accuracy: 0.8220 - val_loss: 0.1945 - val_accuracy: 0.9408 - lr: 0.0250
Epoch 2/25
375/375 [==============================] - ETA: 0s - loss: 0.3432 - accuracy: 0.8961
Epoch 2: val_accuracy did not improve from 0.94075
375/375 [==============================] - 11s 29ms/step - loss: 0.3432 - accuracy: 0.8961 - val_loss: 0.2327 - val_accuracy: 0.9258 - lr: 0.0250
Epoch 3/25
375/375 [==============================] - ETA: 0s - loss: 0.2793 - accuracy: 0.9170
Epoch 3: val_accuracy improved from 0.94075 to 0.96742, saving model to best_model.h5
375/375 [==============================] - 11s 29ms/step - loss: 0.2793 - accuracy: 0.9170 - val_loss: 0.1132 - val_accuracy: 0.9674 - lr: 0.0250
Epoch 4/25
374/375 [============================>.] - 

## Ejercicio:

Probar todas las técnicas presentadas para obtener un acierto en **test > 99%**.

Se aconseja no malgastar datos de entrenamiento y por lo tanto emplear todo el training set para el entrenamiento. No emplear conjunto de validación y emplear el test set al final para calcular el acierto.

A modo de "trampa" podríamos ejecutar el fit con los datos de test en validation_data para así monitorizar si llegamos a ese 99%

validation_data=(x_test, y_test)


In [12]:
# Importar y normalizar datos
(x_train2, y_train2), (x_test, y_test) = mnist.load_data()

print('training set', x_train.shape)
print('test set', x_test.shape)

x_train2 = x_train2.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train2 = x_train2.astype('float32')
x_test = x_test.astype('float32')

# Normalize [0..255]-->[0..1]
x_train2 /= 255
x_test /= 255

# convert class vectors to binary class matrices
num_classes=10
y_train2 = keras.utils.to_categorical(y_train2, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train2, y_train2, test_size=0.2, random_state=42)

print('training set', x_train.shape)
print('val set', x_val.shape)

training set (48000, 28, 28, 1)
test set (10000, 28, 28)
training set (48000, 784)
val set (12000, 784)


In [23]:
## Implementamos en el ejemplo base el aumentado de datos
from keras import Sequential
from keras.layers import Dense, Input, Dropout, BatchNormalization,Reshape
from keras.optimizers import SGD,Adam,AdamW
from keras.callbacks import ReduceLROnPlateau

from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest')

## Importante: ImageDataGenerator espera una imagen con 3 canales, necesitamos hacer reshape
x_train2 = x_train2.reshape(60000, 28, 28, 1)
#x_val = x_val.reshape(12000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)

## Ajustamos el generador de datos
datagen.fit(x_train2)

model = Sequential()

model.add(Input((28,28,1)))
model.add(Reshape((784,)))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

#opt=SGD(learning_rate=0.025, momentum=0.9)
# lr = 0.01
opt = AdamW(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=2, min_lr=0.00001)
checkpoint = keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
#earlystopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3)

epochs=50
batch_size=128
## Entrenamos con el generador de datos en lugar de con el dataset
history = model.fit(datagen.flow(x_train2, y_train2, batch_size=batch_size),
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=[reduce_lr,checkpoint])

## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/50
468/469 [============================>.] - ETA: 0s - loss: 0.8018 - accuracy: 0.7602
Epoch 1: val_accuracy improved from -inf to 0.92280, saving model to best_model.h5
469/469 [==============================] - 18s 36ms/step - loss: 0.8013 - accuracy: 0.7604 - val_loss: 0.2484 - val_accuracy: 0.9228 - lr: 0.0100
Epoch 2/50
468/469 [============================>.] - ETA: 0s - loss: 0.4408 - accuracy: 0.8607
Epoch 2: val_accuracy improved from 0.92280 to 0.95120, saving model to best_model.h5
469/469 [==============================] - 17s 35ms/step - loss: 0.4409 - accuracy: 0.8606 - val_loss: 0.1446 - val_accuracy: 0.9512 - lr: 0.0100
Epoch 3/50
469/469 [==============================] - ETA: 0s - loss: 0.3718 - accuracy: 0.8824 
Epoch 3: val_accuracy improved from 0.95120 to 0.96150, saving model to best_model.h5
469/469 [==============================] - 4652s 10s/step - loss: 0.3718 - accuracy: 0.8824 - val_loss: 0.1166 - val_accuracy: 0.9615 - lr: 0.0100
Epoch 4/50
469/46